In [1]:
import pandas as pd, numpy as np, _pickle as pickle, re, json
from pathlib import Path
from sqlalchemy import create_engine
from time import sleep, time
from tqdm import tqdm
from collections import Counter, defaultdict
from itertools import chain
from bz2 import BZ2File

# with open('../psql_engine.txt') as f:
#     psql = create_engine(f.read())

## tokenize

Turns readability htmls into tokenized text and saves it to database.<br>
Uses simple pretrained logistic regression classifier to remove unrelated items: datetime paragraphs, "read also" paragraphs etc.<br>
Tokenizes Russian text with `nltk` and Ukrainian - using `tokenize_uk`.<br>
Select texts not longer than 8000 characters. Longer texts are analytics or articles, they are written differently and are not considered in this research 

In [2]:
from bs4 import BeautifulSoup
from nltk.tokenize import *
from tokenize_uk.tokenize_uk import tokenize_sents, tokenize_words

Postgresql query to filter incorectly loaded articles.<br>No need to use - sample contains only loaded ones.<br>There are around 5% of incorectly loaded texts

In [16]:
htmls = pd.read_json('../htmls_sample.jl.bz2', lines=True, chunksize=1000)

# bad = ['Руководство сайта не несет ответственности за достоверность материалов, присланных нашими читателями. Администрация сайта, публикуя статьи наших читателей, предупреждает, что их мнения могут не совпадать',
#        'Усі права захищені. Матеріали із сайта',
#        'Все права на материалы, опубликованные на данном ресурсе, принадлежат',
#        'не подлежат дальнейшему воспроизведению и/или распространению в какой-либо форме',
#        'При цитуванні і використанні будь-яких матеріалів в Інтернеті',
#        'Регистрация пользователя в сервисе РИА Клуб на сайте Ria.Ru и авторизация',
#        '© Автономная некоммерческая организация «ТВ-Новости», 2005—2017 гг. Все права защищены', 
#        'Лидер © 2001-2017 UA-Reporter.com Первое информационное интернет-издание Закарпатской области', 
#        'Ваш регион:   Основной сайт Москва Северо-Запад Урал Сибирь',
#        '18+ Настоящий ресурс может содержать материалы 18+ При цитировании информации гиперссылка на ИА',
#        'Введіть слово, щоб почати',
#        'Все об украинской политике, олигархи, которые руководят Украиной, Янукович, украинский парламент, политические новости, выборы, интервью с известными политиками, мнения политтехнологов, новостис регионов, комментарии читателей, ТОП 100 влиятельных украинцев,коррупция в украинской политике, криминал украинской политики,новости Верховного Совета Украины',
#        'Все права защищены Все права на материалы, опубликованные на данном ресурсе, принадлежат ООО',
#        'Любое использование материалов c сайта или программ телеканала 112 Украина разрешается при согласовании с редакцией',
#        'Материалы, содержащие отметку Пресс-релиз, могут быть опубликованы на правах рекламы. Материалы с пометкой',
#        'News24UA - новости Украины, новости политики, самые свежие новости экономики, общества и криминала Новости сегодня в Украине Верховная Рада Украины, новые законопроекты, комментарии украинских политиков и парламентариев',
#        'новости Украины, новости украины сегодня, последние новости украины, новости часа, новости дня, новости онлайн, последние новости в украине',
#        'Похоже, что вы используете блокировщик рекламы :\(Чтобы пользоваться всеми функциями сайта',
#        '- "INSIDER LIFE NEWS" - insiderlifenews@gmail.com 2014 .',
#        '2014-2017 . . \r . \r " \(\) " \r . 34 . " " -',
#        'Copyright © 1999-2018, технология и дизайн принадлежат ООО «Правда.Ру»',
#        'выдано Федеральной службой по надзору в сфере связи, информационных технологий и массовых коммуникаций',
#        'Перепечатка, копирование или воспроизведение информации, опубликованной на сайте',
#        'Введите слово, чтобы начать',
#        'Надежные VPS/VDS, выделенные серверы и хостинг',
#        'влажность: давление: ветер:',
#        'Усі права захищено. Думка автора статті не відображає думку редакції',
#        'Использование материалов и новостей Сегодня разрешается при условии ссылки на Сегодня.ua',
#        'PolitCentr.ru 2013-2017',
#        'Читайте виртуальные журналы RT на русском в Flipboard',
#        'Наша цель – актуальное освещение событий касающихся политической ситуации в Украине и вокруг неё, войны на территории Украины, ситуации сложившейся в Крыму',
#        'Войдите через социальные сети: или авторизуйтесь:',
#        'Мнение редакции может не совпадать с точкой зрения авторов публикаций']

# bad_search_q = '\nor '.join(f'''ra_summary ~~* '%%{cr.replace("'", "''").replace('%', '%%')}%%' '''
#                             for cr in bad)

# q = f'''
# SELECT html_id, ra_title, ra_summary, real_url, link, lang FROM htmls
# WHERE tokenized isnull
#       and not (LENGTH(REGEXP_replace(ra_summary, '[^А-Яа-яІіЇїЄє]', '')) < 20 OR
#               ra_summary = '<html><body/></html>'
#               or ra_summary ~* '[‡ЂЏЎЋѓЃЊµ]'
#               or ra_summary isnull
#               or {bad_search_q}
#       )
#       and LENGTH(REGEXP_replace(ra_summary, '[^А-Яа-яІіЇїЄє]', '')) < 7000;
# '''

# htmls = pd.read_sql(q, psql, chunksize=20000)

Load paragraph classifier - scikit-learn logistic regression, and DictVertorizer for feature transformation.<br>
Features:
* html tag name,
* its classes,
* and id, 
* Length of inner text,
* bag of words of element contents

In [5]:
with open('dvect_tech_tags.pkl', 'rb') as f:
    dv = pickle.load(f)
    
with open('classify_tech_tags.pkl', 'rb') as f:
    cls = pickle.load(f)

In [6]:
def get_feats(tag):
    '''
    returns feature dict for paragraph classifier.
    features: htmls tag name, its classes, and id. Length of inner text and bag of words of element contents.
    '''
    name = tag.name
    
    text = re.sub('[^A-zА-яІіЇїЄєҐґ0-9 ]', ' ', tag.get_text(' ')).split()
    text = [re.sub('[0-9]', '5', str(i)) for i in text]

    class_full = tag.get('class')
    if class_full is not None:
        classes = list(chain(*[re.split('-+|_+', cl) for cl in class_full]))
    else:
        classes = []

    id_full = tag.get('id')
    id_list = re.split('-+|_+', id_full.lower()) if id_full else []
    
    words = [f'word_{w.lower()}' for w in text]
    cls_feats = [f'class_{w}' for w in classes]

    id_feats = [f'id_{w}' for w in id_list]

    feats = {**Counter(cls_feats), **Counter(id_feats), **Counter(words)}
    feats['len'] = len(words)

    feats[f'tagname_{name}'] = 1
    
    return feats

In [7]:
def tokenize_html(row):
    soup = BeautifulSoup(re.sub('<br/?>', '</p><p>', row.ra_summary), 'lxml')
    [t.extract() for t in soup.select('div#more-items-infinite, div.fb-post, p.go_out, div.twitter-tweet')]
    
    [t.extract() for t in soup.find_all('h1', text='Новини по темі')]
    [t.extract() for t in soup.find_all('p', text=re.compile(
        'в даний момент ви читаєте новину|в даный момент вы читаете новость .* на [(eizvestia)|(enovosti)]\.com',
        flags=re.I
    ))]
    url = row.real_url if row.real_url else row.link
    url = url if url else ''
    if 'ria.ru' in url:
        ria_caption = soup.select('div[itemprop="articleBody"] strong')
        if len(ria_caption) != 0: ria_caption[0].extract()

    hs = soup.select('h1, h2, h3, h4, h5')
    paragraphs = [t for t in soup.find_all() 
                  if t.name in ['h1','h2','h3','h4','h5','h6','p', 'div', 'li']
                     and not t.p
                     and not t.div]
    
    intersection = list(filter(lambda t: t in paragraphs[:5], hs))
    if len(intersection) != 0:
        del paragraphs[paragraphs.index(intersection[0])]
        
    if len(paragraphs) == 0: return
    
    X = dv.transform(list(map(get_feats, paragraphs)))
    preds = cls.predict_proba(X)[:, 0]
    paragraps = [p for pred, p in zip(preds.tolist(), paragraphs) if pred > 0.18]    
    paragraphs = [re.sub('\s+', ' ', p.get_text(' ').replace('\xa0', ' ')).strip() for p in paragraphs]
    paragraphs = list(filter(lambda p: len(re.sub('[^A-zА-яІіЇїЄєҐґ]', '', p)) > 2, paragraphs))
    
    if row.lang == 'uk':
        tokenized = '\n\n'.join('\n'.join([' '.join(tokenize_words(s))
                                           for s in tokenize_sents(p)])
                                for p in paragraphs)
    elif row.lang == 'ru':
        tokenized = '\n\n'.join('\n'.join([' '.join(word_tokenize(s))
                                           for s in sent_tokenize(p)])
                                for p in paragraphs)
    return re.sub("``|''", '"', tokenized)

In [ ]:
with BZ2File('tokenized_htmls.jl.bz2', 'w') as f:
    for df in tqdm(htmls):
        df['tokenized'] = df.apply(tokenize_html, axis=1)
        df = df.loc[pd.notnull(df.tokenized)
              ].reindex(['html_id', 'tokenized', 'lang'], axis=1
              ).copy()
        
        f.write(
            (df.to_json(orient='records', lines=True, force_ascii=False) + '\n'
            ).encode('utf-8')
        )
#         # in case of postgres
#     df.to_json('tokenized_htmls.jl.bz2', compression='bz2', )
#     vals = ', '.join([f'''({html_id}, '{tok.replace("'", "''").replace('%', '%%')}')'''
#                       for html_id, tok in df.loc[pd.notnull(df.tokenized)].reindex(['html_id', 'tokenized'], axis=1).values])
#     psql.execute(f'''
#                   update htmls as t set
#                   tokenized = c.tok
#                   from (values
#                       {vals}
#                   ) as c(html_id, tok) 
#                   where c.html_id = t.html_id;
#                   ''')

## str2id

Transforms space delimited tokenized text into an array of word ids according to dictionary. Uses separate dictionaries for Ukrainian and Russian language

In [28]:
tokenized = pd.read_json('tokenized_htmls.jl.bz2', lines=True, chunksize=1000)
# tokenized = pd.read_sql('''
#                         select html_id, tokenized, lang from htmls
#                         where tokenized notnull
#                           and word_ids isnull;
#                         ''', psql, chunksize=10000)

In [29]:
BOS = 'xbos' #beginning of string tag
FLD = 'xbod' #beginning of doc tag

with open('itos_ru.pkl', 'rb') as f:
    itos_ru = pickle.load(f)
    #reverse  - return id for every word, id "0" for out of dictionary tokens 
    stoi_ru = defaultdict(lambda: 0, {v: k for k, v in enumerate(itos_ru)})
with open('itos_uk.pkl', 'rb') as f:
    itos_uk = pickle.load(f)
    stoi_uk = defaultdict(lambda: 0, {v: k for k, v in enumerate(itos_uk)})

`itos` - dictionary for language composed of all loaded news texts. Up to 60000 tokens that occur more than 15 times in all news

In [30]:
def split_tok(text):
    return' \n '.join([p.replace('\n', f' {BOS} ')
                       for p in text.strip().split('\n\n')]
                     ).split(' ')

def tok2id(row):
    stoi = stoi_ru if row.lang == 'ru' else stoi_uk
    return [stoi_ru[token] for token in split_tok(row.tokenized)]

In [ ]:
with BZ2File('word_ids_htmls.jl.bz2', 'w') as f:
    for df in tqdm(tokenized):
        df.tokenized = f'\n{BOS} {FLD} 1 ' + df.tokenized
        # "1" is a mistake, but since it occurs in every article, and at the beginning,
        # I believe it doesn't influence the result - LSTM will forget it by the end of text and will not find it meaningful
        df['word_ids'] = df.apply(tok2id, axis=1)
        df = df.loc[pd.notnull(df.word_ids)
              ].reindex(['html_id', 'word_ids'], axis=1
              ).copy()
        
        f.write(
            (df.to_json(orient='records', lines=True, force_ascii=False) + '\n'
            ).encode('utf-8')
        )
#     # for postgres
#     vals = ', '.join([f'''({html_id}, ARRAY{str2id})'''
#                       for html_id, str2id in df.reindex(['html_id', 'word_ids'], axis=1).values])
#     psql.execute(f'''
#                   update htmls as t set
#                   word_ids = c.word_ids
#                   from (values
#                       {vals}
#                   ) as c(html_id, word_ids) 
#                   where c.html_id = t.html_id;
#                   ''')